**Combine csv files into one hdf5 dataset. Location coordinates are added.**

In [ ]:
!pip install -U tables
import pandas as pd
import numpy as np
import json
import os

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
dataset_root = '/content/gdrive/My Drive/TaxiData'

In [ ]:
cx_map = json.load(open(os.path.join(dataset_root, 'cx.json')))
cy_map = json.load(open(os.path.join(dataset_root, 'cy.json')))
cx_map = {int(k): v for k, v in cx_map.items()}
cy_map = {int(k): v for k, v in cy_map.items()}

In [ ]:
def load_all(root):
  with open(os.path.join(root, 'manifest.txt')) as f:
    files = f.readlines()
  files = map(lambda filename: os.path.join(root, filename.strip()), files)
  frames = list(map(pd.read_csv, files))
  dataset = pd.concat(frames, ignore_index=True)
  dataset['PULocationX'] = dataset['PULocationID'].map(lambda v: cx_map[v])
  dataset['PULocationY'] = dataset['PULocationID'].map(lambda v: cy_map[v])
  dataset['DOLocationX'] = dataset['DOLocationID'].map(lambda v: cx_map[v])
  dataset['DOLocationY'] = dataset['DOLocationID'].map(lambda v: cy_map[v])
  return dataset

In [ ]:
dataset = load_all(os.path.join(dataset_root, 'TaxiData.zip (Unzipped Files)'))
print(dataset)
dataset.to_hdf(os.path.join(dataset_root, 'data.h5'),'df')